# LCT GPT Experiment Workbench
The aim of this notebook is to provide researchers an experiment workbench to finetune and optimise LLM prompts and to evaluate their performance on a set of examples. For this project the LLM (default: OpenAI GPT-3.5) objective is to detect and to identify the sequencing types of combination of passages (clauses). The researcher can use this notebook to optimise LLM response, which is generated based on the following three components (see `Schema and Input data` below):
- LCT definitions (editable): optimised by researcher in LCT definition file (example file: `sequencing_types.xlsx`).
- LCT examples with LCT classes, linkage words and reasoning (editable): examples can be removed, added, or edited by reseacher in example file (example file: `sequencing_examples_2clauses.xlsx`).
- Prompt parsing instuctions (partially editable): Can be optimised, but should be taken with care since correct output parsing instructions are crucial to extract results (file: `instruction_prompt.txt`).

The files are stored in the folder `schemas`. You can find this folder in the parent folder of this notebook. If you edit these files it is recommended to make a copy of the example file first, and to give it a unique name for version control.

## Installation 

Not required if you are running this notebook on Binder. If you run this notebook locally or on your own host environment (e.g. Colab, JupyterLab), you need to install the Python dependencies first (recommend to setup a virtual Python environment first). To install the dependencies, uncomment and run the commands in the following cell.

In [ ]:
# !git clone https://github.com/Sydney-Informatics-Hub/LCT_sequencing
# %cd LCT_sequencing
# %pip install -r requirements.txt

## Import dependencies

This assumes you have a copy of the repo in your parent directory including the folder "tools".

In [ ]:
# Add package libraries in folder tools
import sys
sys.path.insert(1, '..')
from llm.load_schema_json import load_json, validate_json, json_to_dataframe
from llm.utils_llm import LLM, openai_apikey_input
from llm.excel_json_converter import excel_to_json

#to open notebook in interactive mode, uncomment following line
#init_notebook_mode(all_interactive=True)

## Schema and Input data

Define locations of files for sequencing types, instruction prompts, and examples.

In [ ]:
# Path to output folder (if not existing, it will be created)
outpath = "./results/"

# Path to schemas and excel files for definitions and examples:
path_schema = "../schemas/"

# Filename for examples (.json or .xlsx), assume to be in folder path_schema:
filename_examples = "sequencing_examples_2clauses.xlsx"

# Filename for sequencing definitions (.json or .xlsx), assumed to be in folder path_schema:
filename_definitions = "sequencing_types.xlsx"

# if you want to select manually examples from the example table (filename_examples), select indices here as list
# Ideally you want to select at least one example per sequencing type.
# Note that indices count start with 0, not the excel row number, e.g.:
# list_prompt_indices = [0,1,7,14,23,26,30,33]
# Alternatively set None or empty list. In this case a random selection of examples will be used
list_prompt_indices = None

# Filename for prompt instructions
filename_zero_prompt = "instruction_prompt.txt"

# Filename for json schema valdiation ()
filename_schema = "schema_sequencing_examples_reason.json"

## OpenAI Authentication

Authentication with your OpenAI API key for GPT usage (default model: GPT-3.5).
Note that this will incur charges on your OpenAI account.
The widget below allows you to enter your password string using an obfuscated text input box.

In [ ]:
openai_apikey_input()

## Run experiment pipeline

The experiment pipeline includes the following main steps:
- load examples and LCt type definitions from files
- split examples in prompt and test samples
- generate prompt string from LCT type definitions and prompt examples
- call OpenAI API for completion of prompt for each test sample
- save prompt and response to file for each sample
- save combined results to csv file
- evaluate results and save evaluation metrics to file

All output files are saved in the output path folder as specified above.

In [ ]:
import llm.experiment as ex
df_results, outpath_exp, seq_classes = ex.run_pipe(
        outpath = outpath, 
        path_schema = path_schema, 
        filename_examples = filename_examples, 
        filename_schema = filename_schema, 
        filename_definitions = filename_definitions, 
        filename_zero_prompt = filename_zero_prompt, 
        list_prompt_indices = list_prompt_indices)

### Show results table for all samples

The results table (see also`results_gpt-3.5-turbo-instruct.csv`) provides for each sample:
- the string for each test sample (and additional split up in the two text clauses or chunks)
- ground truth classification (test_class)
- ground truth linkage word (test_linkage)
- classification predicted by the model (pred_class)
- probability of the predicted classification (pred_class_prob)
- predicted linkage_word (pred_linkage)
- filenames for prompt and response files of GPT as saved in output directory
- number of total tokens used for each sample
- the modelname and version
- the GPT reasoning for the predicted class



In [ ]:
df_results

### Evaluate and print results stats

This will compute the confusion matrix and the evaluation metrics for the predicted classes by comparing the predicted labels (pred_class) with true labels (test_class). All results are also saved in the output folder. The following metrics are calculated based on the confusion matrix:
- accuracy
- recall
- precision

In [ ]:
ex.eval_exp(df_results, outpath_exp, seq_classes)

## Optional: Zip result exp folder for download

In [ ]:
# zip via shutil
import shutil
import os
shutil.make_archive(os.path.basename(outpath_exp), 'zip', base_dir = outpath_exp)